# 1. Determinación de bases y promedios para tramos

## A. Info a utilizar

* CWTR SUM 
* XML (otro empleador)
* Medidas SAP (IT 0)

## B. Importes consolidados por mes

In [9]:
#importo las librerías que voy a usar

import pandas as pd
from data import CWTR, CWTR_SUM, DOTACION, CUILES
from deducciones.incremental.calculo import *
import empleado as ee

In [10]:
#primero tengo que consolidar la info que traigo de SAP (CWTR_SUM) y el XML (ingresos otro empleador)

#me traigo la info de la CWTR_SUM ya que esta los tiene agrupados por importe por mes
df_rem_mes = pd.DataFrame([], columns=["legajo","Mes", "Importe"])
count = 0
for index, row in DOTACION.iterrows():
    
    # count+=1
    # if count == 50:
    #     break
    legajo = row["legajo"]
    
    
    for m in range(1,13):
        restar_prorrateos = DeduccionEspecial.obtener_rem_mes(legajo,m)
        datos_emp = [[legajo, m, rem_mes]]
        df1 = pd.DataFrame(datos_emp, columns=["legajo","Mes", "Importe"])
        df_rem_mes = pd.concat([df_rem_mes, df1], ignore_index=True) #guardar en el mismo df, el concatenado de ese df (inicialmente vacío) y el que voy creando con cada iteración. En la segunda iteración va tener ese df mas el nuevo y asi sucesivamente. 
display(df_rem_mes)



,legajo,Mes,Importe
0,6000002,1,466593.92
1,6000002,2,1405581.67
2,6000002,3,359751.07
3,6000002,4,363686.19
4,6000002,5,363361.69
...,...,...,...
37171,6109479,8,0.00
37172,6109479,9,0.00
37173,6109479,10,0.00
37174,6109479,11,0.00


In [11]:
#leo el df 572 parquet
df_572 = pd.read_parquet("../ganancias-cuarta-categoria/572 web/572web.parquet")
df_572.to_clipboard()

In [ ]:
# dejo las columnas que necesito
filtro = df_572["Descripción"].eq("OE-ganBrut")
df_filtrado = df_572[filtro]
df_filtrado = df_filtrado[["Cuil", "Mes", "Importe"]].reset_index(drop=True)
#paso la columna cuil a int, ya que si no el cruce me da error
df_filtrado["Cuil"]=df_filtrado["Cuil"].astype("int64")
#agrupo por importe los ingresos de otro empleador
df_base_OE = df_filtrado.groupby(['Cuil','Mes'])['Importe'].sum().reset_index() 
df_base_OE

,Cuil,Mes,Importe
0,23165012879,1,80507.67
1,23165012879,2,80507.67
2,23165012879,3,80507.67
3,23165012879,4,80507.67
4,23165012879,5,80507.67
5,23165012879,6,135663.00
6,23165012879,7,80507.67
7,23165012879,8,80507.67
8,23165012879,9,149698.56
9,23165012879,10,149698.56


In [17]:
#como solo tengo el dato del cuil, curzo con el archivo CUILES y me traigo el legajo
df_oe_unido = df_base_OE.merge(CUILES, how='left', left_on="Cuil", right_on="cuil")
df_oe_unido = df_oe_unido[["legajo", "Mes", "Importe"]].reset_index(drop=True)

#luego de que ya tengo el legajo, concateno con el dataframe de la cwtr y ahora tengo el total de rem por mes
df_rem_mes_finales = pd.concat([df_rem_mes, df_oe_unido], ignore_index=True)



In [18]:
#lo guardo en un Excel
df_rem_mes_finales.to_excel("bases_por_mes.xlsx")

## C. Cálculo de promedios a cada mes 

In [ ]:
df_promedios = pd.DataFrame()
count = 0
for index, row in df_rem_mes_finales.iterrows():
    
    # count+=1
    # if count == 30:
    #     break
    legajo = row["legajo"]
    mes = row["Mes"]
    importe = row["Importe"]
    
    norma = DeduccionEspecial.norma_aplicable(mes)
    valor_promedio = DeduccionEspecial.obtener_promedio_rem_df(legajo, mes, norma, df_rem_mes_finales)
    base_tramo = min(importe,valor_promedio)
    
    tramo = DeduccionEspecial.obtener_tramo(base_tramo, mes)

    datos_emp = [[legajo, mes, importe, valor_promedio, base_tramo, tramo]]
    df1 = pd.DataFrame(datos_emp, columns=["legajo","mes", "Importe", "promedio", "base_tramo", "tramo"])
    df_promedios = pd.concat([df_promedios, df1], ignore_index=True) #guardar en el mismo df, el concatenado de ese df (inicialmente vacío) y el que voy creando con cada iteración. En la segunda iteración va tener ese df mas el nuevo y asi sucesivamente. 
display(df_promedios)

In [21]:
#se deja la base guardada en middle para despues ser utilizada en otros cálculos 
df_promedios.to_parquet('../bases-ganancias-2022/middle/bases_tramos.parquet', compression='brotli')
